In [1]:
using LinearAlgebra
using TensorKit
using KrylovKit
using JSON
using HDF5, JLD
cd("D:\\My Documents\\Code\\Julia_codes\\Tensor network\\IPEPS_TensorKit\\kagome\\SU2_PG")
#push!(LOAD_PATH, "D:\\My Documents\\Code\\Julia_codes\\Tensor network\\IPEPS_TensorKit\\kagome\\SU2_PG")
include("kagome_load_tensor.jl")
include("kagome_CTMRG.jl")
include("kagome_model.jl")
include("kagome_IPESS.jl")
include("mps_algorithms\\ITEBD_algorithms.jl")
include("mps_algorithms\\TransfOp_decomposition.jl")
include("mps_algorithms\\PUMPS_algorithms.jl")

D=8;
chi=20;
W=20;
N=20;
tol=1e-6;
EH_n=3;#number of entanglement spectrum per k point

Dtrun_init=200;
Dtrun_max=200;
trun_tol=1e-8;
group_size=Int(round((10^8)/(chi*chi*W*W*D)));

mpo_type="O_O";#"O_O" or "OO"

pow=Int((N-2)/2);

J1=1;
J2=0;
J3=0;
Jchi=0;
Jtrip=0;


In [2]:

parameters=Dict([("J1", J1), ("J2", J2), ("J3", J3), ("Jchi", Jchi), ("Jtrip", Jtrip)]);

A_set,B_set,A1_set,A2_set, A_set_occu,B_set_occu,A1_set_occu,A2_set_occu, S_label, Sz_label, virtual_particle, Va, Vb=construct_tensor(D);

filenm="LS_D_"*string(D)*"_chi_40.json"
json_dict=read_json_state(filenm);

bond_tensor,triangle_tensor=construct_su2_PG_IPESS(json_dict,A_set,B_set,A1_set,A2_set, A_set_occu,B_set_occu,A1_set_occu,A2_set_occu, S_label, Sz_label, virtual_particle, Va, Vb);

PEPS_tensor=bond_tensor;
@tensor PEPS_tensor[:] := bond_tensor[-1,1,-5]*bond_tensor[4,3,-6]*bond_tensor[-4,2,-7]*triangle_tensor[1,3,2]*triangle_tensor[4,-2,-3];
A_unfused=PEPS_tensor;

U_phy=unitary(fuse(space(PEPS_tensor, 5) ⊗ space(PEPS_tensor, 6) ⊗ space(PEPS_tensor, 7)), space(PEPS_tensor, 5) ⊗ space(PEPS_tensor, 6) ⊗ space(PEPS_tensor, 7));
@tensor A_fused[:] :=PEPS_tensor[-1,-2,-3,-4,1,2,3]*U_phy[-5,1,2,3];


In [3]:
CTM=[];
U_L=[];
U_D=[];
U_R=[];
U_U=[];

try
    #load data
    jld_filenm="CTM_D"*string(D)*"_chi"*string(chi)*".jld";
    CTM_dict=load(jld_filenm)["CTM_dict"];
    U_L_dict=load(jld_filenm)["U_L_dict"];
    U_D_dict=load(jld_filenm)["U_D_dict"];
    U_R_dict=load(jld_filenm)["U_R_dict"];
    U_U_dict=load(jld_filenm)["U_U_dict"];
    CTM=deepcopy(CTM_dict)
    for cc=1:4
        CTM["Tset"][cc]=convert(TensorMap,CTM_dict["Tset"][cc]);
        CTM["Cset"][cc]=convert(TensorMap,CTM_dict["Cset"][cc]);
    end
    U_L=convert(TensorMap,U_L_dict);
    U_D=convert(TensorMap,U_D_dict);
    U_R=convert(TensorMap,U_R_dict);
    U_U=convert(TensorMap,U_U_dict);

    display("load CTM from saved data directly")
catch e

    display("No CTM found from saved data, now do CTMRG")


    init=Dict([("CTM", []), ("init_type", "PBC")]);
    conv_check="singular_value";
    @time CTM, AA_fused, U_L,U_D,U_R,U_U=CTMRG(A_fused,chi,conv_check,tol,init);
    
    @time E_up, E_down=evaluate_ob(parameters, U_phy, A_unfused, A_fused, AA_fused, U_L,U_D,U_R,U_U, CTM, "E_triangle");
    @time E_up_12, E_up_31, E_up_23, E_down_12, E_down_31, E_down_23=evaluate_ob(parameters, U_phy, A_unfused, A_fused, AA_fused, U_L,U_D,U_R,U_U, CTM, "E_bond");
    display((E_up+E_down)/3)

    
    CTM_dict=deepcopy(CTM)
    for cc=1:4
        CTM_dict["Tset"][cc]=convert(Dict,CTM_dict["Tset"][cc]);
        CTM_dict["Cset"][cc]=convert(Dict,CTM_dict["Cset"][cc]);
    end
    U_L_dict=convert(Dict,U_L);
    U_D_dict=convert(Dict,U_D);
    U_R_dict=convert(Dict,U_R);
    U_U_dict=convert(Dict,U_U);

    jld_filenm="CTM_D"*string(D)*"_chi"*string(chi)*".jld";
    save(jld_filenm, "CTM_dict",CTM_dict, "U_L_dict",U_L_dict,"U_D_dict",U_D_dict,"U_R_dict",U_R_dict,"U_U_dict",U_U_dict);



end


"load CTM from saved data directly"

In [4]:

O1=[];
O2=[];
Ag=[];
try
    #load data
    jld_filenm="itebd_ground_D"*string(D)*"_chi"*string(chi)*"_W"*string(W)*".jld";
    Ag_dict=load(jld_filenm)["Ag_dict"];
    O1_dict=load(jld_filenm)["O1_dict"];
    O2_dict=load(jld_filenm)["O2_dict"];
    Ag=convert(TensorMap,Ag_dict);
    O1=convert(TensorMap,O1_dict);
    O2=convert(TensorMap,O2_dict);
    display("load itebd ground state from saved data directly")
catch e

    display("No itebd ground state found from saved data, now do itebd")
    
    Tleft=CTM["Tset"][4];
    Tright=CTM["Tset"][2];
    @tensor O1[:]:=Tleft[-3,1,-1]*U_L[1,-2,-4];
    @tensor O2[:]:=Tright[-1,1,-3]*U_R[-4,-2,1];
    
    mps_virtual=SU₂Space(0=>1,1/2=>1,1=>1);mps_phy=space(O1,2);
    A_init=permute(TensorMap(randn, mps_virtual*mps_virtual', mps_phy),(1,2,3,),());

    #Ag,A_init=ITEBD_boundary_groundstate(O1,O2,W,A_init,"OO");
    Ag,A_init=ITEBD_boundary_groundstate(O1,O2,W,A_init,"O_O");

    #save itebd data
    O1_dict=convert(Dict,O1);
    O2_dict=convert(Dict,O2);
    Ag_dict=convert(Dict,Ag);
    jld_filenm="itebd_ground_D"*string(D)*"_chi"*string(chi)*"_W"*string(W)*".jld";
    save(jld_filenm, "O1_dict", O1_dict,"O2_dict", O2_dict,"Ag_dict", Ag_dict);

    #save initial CTM to compare with other codes
    matwrite("itebd_matfile.mat", Dict(
        "O1" => convert(Array,O1),
        "O2" => convert(Array,O2),
        "A_init" => convert(Array,A_init),
        "Ag" => convert(Array,Ag)
    ); compress = false)


end

"load itebd ground state from saved data directly"

In [5]:

space_AOA=fuse(space(Ag,1)'⊗space(O2,1)'⊗space(O1,1)⊗ space(Ag,1));
space_AA=fuse(space(Ag,1)'⊗ space(Ag,1));

AOA_sec=collect(sectors(space_AOA))
AA_sec=collect(sectors(space_AA))

@tensor OO[:]:=O1[-2,-3,-5,1]*O2[-1,1,-4,-6];
U_fuse_chichi=unitary(fuse(space(OO,1)⊗ space(OO,2)),space(OO,1)⊗ space(OO,2));
@tensor OO[:]:=U_fuse_chichi[-1,1,2]*OO[1,2,-2,3,4,-4]*U_fuse_chichi'[3,4,-3];
display(space(OO))

(Rep[SU₂](0=>45, 1/2=>60, 1=>45, 3/2=>20, 2=>4) ⊗ Rep[SU₂](0=>1, 1/2=>2, 1=>1) ⊗ Rep[SU₂](0=>45, 1/2=>60, 1=>45, 3/2=>20, 2=>4)' ⊗ Rep[SU₂](0=>1, 1/2=>2, 1=>1)') ← ProductSpace{GradedSpace{SU2Irrep, TensorKit.SortedVectorDict{SU2Irrep, Int64}}, 0}()

In [6]:
# for s in sectors(space_AOA)
#   display(s)
#   @show s, dim(space_AOA, s), dim(s)
# end


In [7]:
#normalize the MPO
euL_set,_,_,_,_=FLR_eig(Ag,OO,20,space_AOA,AOA_sec);
norm_coe=maximum(abs.(group_numbers(euL_set)));
OO=OO/norm_coe;
O1=O1/norm_coe;


In [8]:

check_complete_decomp=false;
if check_complete_decomp
    Dtrun=W*W*chi*chi;

    euL_set,evL_set,euR_set,evR_set,SPIN_eig_set=FLR_eig(Ag,OO,Dtrun,space_AOA,AOA_sec);
    display(size(euL_set))
    eul_set,evl_set,eur_set,evr_set,spin_eig_set=GLR_eig(Ag,Dtrun,space_AA,AA_sec);
    display(size(eul_set))
    S_set,U_set,Vh_set,SPIN_svd_set=FLR_svd(Ag,OO,pow,Dtrun,space_AOA,AOA_sec);
    display(size(S_set))
    s_set,u_set,vh_set,spin_svd_set=GLR_svd(Ag,pow,Dtrun,space_AA,AA_sec);
    display(size(s_set))


    euR_set,evL_set,evR_set,SPIN_eig_set=truncate_sectors(Dtrun,euR_set,evL_set,evR_set,SPIN_eig_set);
    display(euR_set)
    eur_set,evl_set,evr_set,spin_eig_set=truncate_sectors(Dtrun,eur_set,evl_set,evr_set,spin_eig_set);
    display(eur_set)
    S_set,U_set,Vh_set,SPIN_svd_set=truncate_sectors(Dtrun,S_set,U_set,Vh_set,SPIN_svd_set);
    display(S_set)
    s_set,u_set,vh_set,spin_svd_set=truncate_sectors(Dtrun,s_set,u_set,vh_set,spin_svd_set);
    display(s_set)




    display("check FLR_eig")
    @tensor M[:]:=Ag'[-1,-4,1]*OO[-2,2,-5,1]*Ag[-3,-6,2];
    M=permute(M,(1,2,3,),(4,5,6,));
    M0=M;
    for cc=1:pow-1
        M=M*M0;
    end
    M_temp=M*0;
    for cc=1:length(euR_set)
        euR=euR_set[cc];
        evR=evR_set[cc];
        evL=evL_set[cc];
        spin=SPIN_eig_set[cc]
        for cs=1:length(euR)
            @tensor coe=evR[cs][2,3,4,1]*evL[cs][1,2,3,4];
            M_temp=M_temp+euR[cs]^(pow)*evR[cs]*evL[cs]*(spin[cs]*2+1)/coe;
        end
    end
    display(norm(M-M_temp)/norm(M))

    euR_set_combined,evL_set_combined,evR_set_combined,SPIN_eig_set_combined=combine_singlespin_sector(euR_set,evL_set,evR_set,SPIN_eig_set,true)
    M_temp=M*0;
    for cc=1:length(euR_set)
        euR=euR_set_combined[cc];
        evR=evR_set_combined[cc];
        evL=evL_set_combined[cc];
        spin=SPIN_eig_set_combined[cc]
        if euR==[]
        else
            euR0=euR
            for cc=1:pow-1
                euR=euR*euR0;
            end

            M_temp=M_temp+evR*euR*evL*(spin*2+1);
        end
    end
    display(norm(M-M_temp)/norm(M))




    display("check GLR_eig")
    @tensor M[:]:=Ag'[-1,-3,1]*Ag[-2,-4,1];
    M=permute(M,(1,2,),(3,4,));
    M0=M;
    for cc=1:pow-1
        M=M*M0;
    end
    eur_set_combined,evl_set_combined,evr_set_combined,spin_eig_set_combined=combine_singlespin_sector(eur_set,evl_set,evr_set,spin_eig_set,true)

    M_temp=M*0;
    for cc=1:length(eul_set)
        eur=eur_set_combined[cc];
        evl=evl_set_combined[cc];
        evr=evr_set_combined[cc];
        spin=spin_eig_set_combined[cc]
        if eur==[]
        else
            eur0=eur;
            for cc=1:pow-1
                eur=eur*eur0;
            end
            M_temp=M_temp+evr*eur*evl*(spin*2+1);
        end
    end
    display(norm(M-M_temp)/norm(M))





    display("check FLR_svd")
    @tensor M[:]:=Ag'[-1,-4,1]*OO[-2,2,-5,1]*Ag[-3,-6,2];
    M=permute(M,(1,2,3,),(4,5,6,));
    M0=M;
    for cc=1:pow-1
        M=M*M0;
    end
    M_temp=M*0;
    for cc=1:length(S_set)
        S=S_set[cc];
        U=U_set[cc];
        Vh=Vh_set[cc];
        spin=SPIN_svd_set[cc]
        for cs=1:length(S)
            M_temp=M_temp+S[cs]*U[cs]*Vh[cs]*(spin[cs]*2+1);
        end
    end
    display(norm(M-M_temp)/norm(M))

    S_set_combined,Vh_set_combined,U_set_combined,SPIN_svd_set_combined=combine_singlespin_sector(S_set,Vh_set,U_set,SPIN_svd_set,false)
    M_temp=M*0;
    for cc=1:length(S_set)
        S=S_set_combined[cc];
        U=U_set_combined[cc];
        Vh=Vh_set_combined[cc];
        spin=SPIN_svd_set_combined[cc]
        if S==[]
        else
            M_temp=M_temp+U*S*Vh*(spin*2+1);
        end
    end
    display(norm(M-M_temp)/norm(M))



    display("check GLR_svd")
    @tensor M[:]:=Ag'[-1,-3,1]*Ag[-2,-4,1];
    M=permute(M,(1,2,),(3,4,));
    M0=M;
    for cc=1:pow-1
        M=M*M0;
    end
    M_temp=M*0;
    for cc=1:length(s_set)
        s=s_set[cc];
        u=u_set[cc];
        vh=vh_set[cc];
        spin=spin_svd_set[cc]
        for cs=1:length(s)
            M_temp=M_temp+s[cs]*u[cs]*vh[cs]*(spin[cs]*2+1);
        end
    end
    display(norm(M-M_temp)/norm(M))

    s_set_combined,vh_set_combined,u_set_combined,spin_svd_set_combined=combine_singlespin_sector(s_set,vh_set,u_set,spin_svd_set,false)


    M_temp=M*0;
    for cc=1:length(s_set)
        s=s_set_combined[cc];
        vh=vh_set_combined[cc];
        u=u_set_combined[cc];
        spin=spin_svd_set_combined[cc]
        if s==[]
        else
            M_temp=M_temp+u*s*vh*(spin*2+1);
        end
    end
    display(norm(M-M_temp)/norm(M))

end


In [9]:
euR_set,evL_set,evR_set,SPIN_eig_set=TransfOp_decom(Ag,OO,space_AOA,AOA_sec,pow,Dtrun_init,Dtrun_max,trun_tol,"eigenvalue_FLR");
# display(euR_set)

eur_set,evl_set,evr_set,spin_eig_set=TransfOp_decom(Ag,OO,space_AA,AA_sec,pow,Dtrun_init,Dtrun_max,trun_tol,"eigenvalue_GLR");
# display(eur_set)

"FLR_eig, DTrun=200, SU2 reduced to 91, minimal eigenvalue: 1.220763939968997e-5"

"GLR_eig, Dtrun=200, SU2 reduced to 78, minimal eigenvalue: 5.717377690650657e-9"

(Any[ComplexF64[1.000000000000002 + 1.1102230246251565e-16im, 0.2508827376560169 + 0.7268994841636415im, 0.2508827376560171 - 0.7268994841636411im, 0.5707986934924273 - 2.693711563412457e-16im, 0.11417801976956217 - 0.33645765073508427im, 0.1141780197695616 + 0.33645765073508344im, -0.23482729967469249 - 0.2168760299626249im, -0.23482729967469246 + 0.21687602996262453im, 0.21568892529805114 - 0.14451949989595256im, 0.2156889252980506 + 0.14451949989595186im, 0.2540403080678695 - 9.473447610561439e-17im, 0.05766954743163173 - 0.2002448579689437im, 0.05766954743163167 + 0.20024485796894345im, 0.14156482625155112 - 0.09121451518477626im, 0.14156482625155084 + 0.09121451518477643im, 0.15020523226594498 - 2.0507367778702766e-18im, 0.06219877031626901 - 0.13641344678946438im, 0.062198770316269046 + 0.13641344678946413im], ComplexF64[0.7692055171350469 + 0.264092005374448im, 0.7692055171350456 - 0.26409200537444594im, 0.3669551719788572 - 0.5195294730459564im, 0.3669551719788558 + 0.519529473

In [10]:

S_set,U_set,Vh_set,SPIN_svd_set=TransfOp_decom(Ag,OO,space_AOA,AOA_sec,pow,Dtrun_init,Dtrun_max,trun_tol,"svd_FLR");
# display(S_set)

s_set,u_set,vh_set,spin_svd_set=TransfOp_decom(Ag,OO,space_AA,AA_sec,pow,Dtrun_init,Dtrun_max,trun_tol,"svd_GLR");
# display(s_set)

"minimal singular value in this sector is quite small, skip checking number of converged values"

"minimal singular value in this sector is quite small, skip checking number of converged values"

"minimal singular value in this sector is quite small, skip checking number of converged values"

"minimal singular value in this sector is quite small, skip checking number of converged values"

"FLR_svd, DTrun=200, SU2 reduced to 94, minimal eigenvalue: 5.8815337471676165e-5"

...

"minimal singular value in this sector is quite small, skip checking number of converged values"

"GLR_svd, Dtrun=200, SU2 reduced to 79, minimal eigenvalue: 7.246538708770393e-8"

Number of singular values obtained are not enough, use smaller tol
Number of singular values obtained are not enough, use smaller tol
Number of singular values obtained are not enough, use smaller tol
Number of singular values obtained are not enough, use smaller tol
Number of singular values obtained are not enough, use smaller tol
Number of singular values obtained are not enough, add noise, increase Krylov subspace and use smaller tol
Number of singular values obtained are not enough, add noise, increase Krylov subspace and use smaller tol
Number of singular values obtained are not enough, add noise, increase Krylov subspace and use smaller tol
Number of singular values obtained are not enough, add noise, increase Krylov subspace and use smaller tol


┌ Warning: Invariant subspace of dimension 18 (up to requested tolerance `tol = 1.0e-12`), which is smaller than the number of requested singular values (i.e. `howmany == 30`); setting `howmany = 18`.
└ @ KrylovKit C:\Users\Lenovo\.julia\packages\KrylovKit\YPiz7\src\eigsolve\svdsolve.jl:142
┌ Warning: number of values converged is not enough
└ @ Main d:\My Documents\Code\Julia_codes\Tensor network\IPEPS_TensorKit\kagome\SU2_PG\mps_algorithms\TransfOp_decomposition.jl:406
┌ Warning: Invariant subspace of dimension 1 (up to requested tolerance `tol = 1.0e-12`), which is smaller than the number of requested singular values (i.e. `howmany == 26`); setting `howmany = 1`.
└ @ KrylovKit C:\Users\Lenovo\.julia\packages\KrylovKit\YPiz7\src\eigsolve\svdsolve.jl:142
┌ Warning: Invariant subspace of dimension 1 (up to requested tolerance `tol = 1.0e-14`), which is smaller than the number of requested singular values (i.e. `howmany == 26`); setting `howmany = 1`.
└ @ KrylovKit C:\Users\Lenovo\.juli

(Any[[2.292882683525971, 0.8821396806767997, 0.20587184314777984, 0.1356775622120479, 0.010851091795518615, 0.00029110652446749115, 9.3703147881379e-5, 7.554714175750199e-5, 6.906145536539553e-5, 1.994807642223695e-5, 7.233962818197525e-6, 5.874699706760304e-6, 4.968871879668593e-6, 1.6656908268337961e-6, 1.0371832331145858e-6], [0.37102808167165563, 0.37102808167165485, 0.3653065226115117, 0.2985539825416451, 0.22045570244737867, 0.02400571317456104, 0.024005713174561034, 0.01158323766275726, 0.011583237662757252, 0.0021767714338136166  …  6.246014866625297e-5, 8.053527017648574e-6, 8.053527017645742e-6, 7.918413059632786e-6, 6.702314152415319e-6, 6.702314152413214e-6, 3.642863498116056e-6, 3.642863498115319e-6, 1.096110576541809e-6, 1.0961105765406495e-6], [0.14882662358082768, 0.1231169053890634, 0.09631472137677644, 0.08348801951397053, 0.06844873244297951, 0.00234231217466997, 0.0006789758310838403, 0.00041900914488134957, 0.00028747417205822283, 0.0001618594631879828  …  2.348816

In [11]:
check_truncated_decomp_error=false;

if mpo_type=="O_O"
    OO_transform=true;
elseif mpo_type=="OO"
    OO_transform=false;
end

euR_set_combined,evL_set_combined,evR_set_combined,SPIN_eig_set_combined=combine_singlespin_sector(euR_set,evL_set,evR_set,SPIN_eig_set,true);
euR_set_grouped,evL_set_grouped,evR_set_grouped,SPIN_eig_set_grouped,DTrun_FLR_eig=group_singlespin_sector(group_size,euR_set_combined,evL_set_combined,evR_set_combined,SPIN_eig_set_combined,OO_transform,U_fuse_chichi)
display("group information:")
display(DTrun_FLR_eig)

eur_set_combined,evl_set_combined,evr_set_combined,spin_eig_set_combined=combine_singlespin_sector(eur_set,evl_set,evr_set,spin_eig_set,true)
eur_set_grouped,evl_set_grouped,evr_set_grouped,spin_eig_set_grouped,Dtrun_GLR_eig=group_singlespin_sector(group_size,eur_set_combined,evl_set_combined,evr_set_combined,spin_eig_set_combined,false,[])
display("group information:")
display(Dtrun_GLR_eig)


S_set_combined,Vh_set_combined,U_set_combined,SPIN_svd_set_combined=combine_singlespin_sector(S_set,Vh_set,U_set,SPIN_svd_set,false)
S_set_grouped,Vh_set_grouped,U_set_grouped,SPIN_svd_set_grouped,DTrun_FLR_svd=group_singlespin_sector(group_size,S_set_combined,Vh_set_combined,U_set_combined,SPIN_svd_set_combined,OO_transform,U_fuse_chichi)
display("group information:")
display(DTrun_FLR_svd)

s_set_combined,vh_set_combined,u_set_combined,spin_svd_set_combined=combine_singlespin_sector(s_set,vh_set,u_set,spin_svd_set,false)
s_set_grouped,vh_set_grouped,u_set_grouped,spin_svd_set_grouped,Dtrun_GLR_svd=group_singlespin_sector(group_size,s_set_combined,vh_set_combined,u_set_combined,spin_svd_set_combined,false,[])
display("group information:")
display(Dtrun_GLR_svd)



if check_truncated_decomp_error

    display("%%%%%%%%%%%%%%%%%%%%%%")
    display("check FLR_eig")
    @tensor M[:]:=Ag'[-1,-4,1]*OO[-2,2,-5,1]*Ag[-3,-6,2];
    M=permute(M,(1,2,3,),(4,5,6,));
    M0=M;
    for cc=1:pow-1
        M=M*M0;
    end

    M_temp=M*0;
    for cc=1:length(euR_set)
        euR=euR_set_combined[cc];
        evR=evR_set_combined[cc];
        evL=evL_set_combined[cc];
        spin=SPIN_eig_set_combined[cc]
        if euR==[]
        else
            euR0=euR
            for ccc=1:pow-1
                euR=euR*euR0;
            end
            M_temp=M_temp+evR*euR*evL*(spin*2+1);
        end
    end
    display(norm(M-M_temp)/norm(M))

    M_temp1=M_temp*0;
    for cc=1:length(euR_set)
        euRs=euR_set_grouped[cc];
        evRs=evR_set_grouped[cc];
        evLs=evL_set_grouped[cc];
        spin=SPIN_eig_set_grouped[cc]
        if euRs==[]
        else
            for cs=1:length(euRs)
                euR0=euRs[cs];
                euR=euR0;
                for ccc=1:pow-1
                    euR=euR*euR0;
                end
                @tensor evR[:]:=U_fuse_chichi[-2,1,2]*evRs[cs][-1,1,2,-3,-4];
                evR=permute(evR,(1,2,3,),(4,));
                @tensor evL[:]:=evLs[cs][-1,-2,1,2,-4]*U_fuse_chichi'[1,2,-3];
                evL=permute(evL,(1,),(2,3,4,));
                M_temp1=M_temp1+evR*euR*evL*(spin*2+1);
            end
        end
    end

    display(norm(M-M_temp1)/norm(M))



    display("%%%%%%%%%%%%%%%%%%%%%%")
    display("check GLR_eig")
    @tensor M[:]:=Ag'[-1,-3,1]*Ag[-2,-4,1];
    M=permute(M,(1,2,),(3,4,));
    M0=M;
    for cc=1:pow-1
        M=M*M0;
    end

    M_temp=M*0;
    for cc=1:length(eur_set)
        eur=eur_set_combined[cc];
        evl=evl_set_combined[cc];
        evr=evr_set_combined[cc];
        spin=spin_eig_set_combined[cc]
        if eur==[]
        else
            eur0=eur;
            for ccc=1:pow-1
                eur=eur*eur0;
            end
            M_temp=M_temp+evr*eur*evl*(spin*2+1);
        end
    end
    display(norm(M-M_temp)/norm(M))

    M_temp1=M_temp*0;
    for cc=1:length(eur_set)
        eurs=eur_set_grouped[cc];
        evrs=evr_set_grouped[cc];
        evls=evl_set_grouped[cc];
        spin=spin_eig_set_grouped[cc]
        if eurs==[]
        else
            for cs=1:length(eurs)
                eur0=eurs[cs];
                eur=eur0;
                for ccc=1:pow-1
                    eur=eur*eur0;
                end

                M_temp1=M_temp1+evrs[cs]*eur*evls[cs]*(spin*2+1);
            end
        end
    end
    display(norm(M-M_temp1)/norm(M))




    display("%%%%%%%%%%%%%%%%%%%%%%")
    display("check FLR_svd")
    @tensor M[:]:=Ag'[-1,-4,1]*OO[-2,2,-5,1]*Ag[-3,-6,2];
    M=permute(M,(1,2,3,),(4,5,6,));
    M0=M;
    for cc=1:pow-1
        M=M*M0;
    end
    M_temp=M*0;
    for cc=1:length(S_set)
        S=S_set[cc];
        U=U_set[cc];
        Vh=Vh_set[cc];
        spin=SPIN_svd_set[cc]
        for cs=1:length(S)
            M_temp=M_temp+S[cs]*U[cs]*Vh[cs]*(spin[cs]*2+1);
        end
    end
    display(norm(M-M_temp)/norm(M))

    M_temp=M*0;
    for cc=1:length(S_set)
        S=S_set_combined[cc];
        U=U_set_combined[cc];
        Vh=Vh_set_combined[cc];
        spin=SPIN_svd_set_combined[cc]
        if S==[]
        else
            M_temp=M_temp+U*S*Vh*(spin*2+1);
        end
    end
    display(norm(M-M_temp)/norm(M))

    M_temp1=M_temp*0;
    for cc=1:length(S_set)
        Ss=S_set_grouped[cc];
        Us=U_set_grouped[cc];
        Vhs=Vh_set_grouped[cc];
        spin=SPIN_svd_set_grouped[cc]
        if Ss==[]
        else
            for cs=1:length(Ss)
                S=Ss[cs];
                @tensor U[:]:=U_fuse_chichi[-2,1,2]*Us[cs][-1,1,2,-3,-4];
                U=permute(U,(1,2,3,),(4,));
                @tensor Vh[:]:=Vhs[cs][-1,-2,1,2,-4]*U_fuse_chichi'[1,2,-3];
                Vh=permute(Vh,(1,),(2,3,4,));
                M_temp1=M_temp1+U*S*Vh*(spin*2+1);
            end
        end
    end
    display(norm(M-M_temp1)/norm(M))



    display("%%%%%%%%%%%%%%%%%%%%%%")
    display("check GLR_svd")
    @tensor M[:]:=Ag'[-1,-3,1]*Ag[-2,-4,1];
    M=permute(M,(1,2,),(3,4,));
    M0=M;
    for cc=1:pow-1
        M=M*M0;
    end
    M_temp=M*0;
    for cc=1:length(s_set)
        s=s_set[cc];
        u=u_set[cc];
        vh=vh_set[cc];
        spin=spin_svd_set[cc]
        for cs=1:length(s)
            M_temp=M_temp+s[cs]*u[cs]*vh[cs]*(spin[cs]*2+1);
        end
    end
    display(norm(M-M_temp)/norm(M))

    M_temp=M*0;
    for cc=1:length(s_set)
        s=s_set_combined[cc];
        vh=vh_set_combined[cc];
        u=u_set_combined[cc];
        spin=spin_svd_set_combined[cc]
        if s==[]
        else
            M_temp=M_temp+u*s*vh*(spin*2+1);
        end
    end
    display(norm(M-M_temp)/norm(M))

    M_temp1=M_temp*0;
    for cc=1:length(s_set)
        ss=s_set_grouped[cc];
        us=u_set_grouped[cc];
        vhs=vh_set_grouped[cc];
        spin=spin_svd_set_grouped[cc]
        if ss==[]
        else
            for cs=1:length(ss)
                s=ss[cs];
                M_temp1=M_temp1+us[cs]*s*vhs[cs]*(spin*2+1);
            end
        end
    end

    display(norm(M-M_temp1)/norm(M))
end

"group information:"

11-element Vector{Any}:
 UnitRange{Int64}[1:25]
 UnitRange{Int64}[1:34]
 UnitRange{Int64}[1:21]
 UnitRange{Int64}[1:11]
 ComplexF64[]
 ComplexF64[]
 ComplexF64[]
 ComplexF64[]
 ComplexF64[]
 ComplexF64[]
 ComplexF64[]

"group information:"

7-element Vector{Any}:
 UnitRange{Int64}[1:18]
 UnitRange{Int64}[1:20]
 UnitRange{Int64}[1:21]
 UnitRange{Int64}[1:10]
 UnitRange{Int64}[1:7]
 ComplexF64[]
 ComplexF64[]

"group information:"

11-element Vector{Any}:
 UnitRange{Int64}[1:28]
 UnitRange{Int64}[1:35]
 UnitRange{Int64}[1:23]
 UnitRange{Int64}[1:8]
 Float64[]
 Float64[]
 Float64[]
 Float64[]
 Float64[]
 Float64[]
 Float64[]

"group information:"

7-element Vector{Any}:
 UnitRange{Int64}[1:15]
 UnitRange{Int64}[1:24]
 UnitRange{Int64}[1:26]
 UnitRange{Int64}[1:11]
 UnitRange{Int64}[1:3]
 Float64[]
 Float64[]

In [12]:
Dtrun_method="svds";
ES_sectors=[0,1/2,1,3/2,2,5/2];
kset=0:N-1;
#kset=0:0
Eset=[];
Trun_err=0;
DTrun=0;
print("calculate ES for N="*string(N));
display("kset="*string(kset));
pow=round((N-2)/2);



if Dtrun_method=="eigs"
    DTrun=length(group_numbers(SPIN_eig_set));
    display("DTrun="*string(DTrun));

    euRs=abs.(group_numbers(euR_set));
    Trun_err=(minimum(euRs)/maximum(euRs))^pow;


    euR_pow=deepcopy(euR_set_grouped);
    for ca=1:length(euR_pow)
        for cb=1:length(euR_pow[ca])
            euR_pow[ca][cb]=euR_pow[ca][cb]^Int(pow);
        end
    end

    kset,Eset=solve_ITEBD_excitation_TrunTransOp_iterative(Ag,O1,O2,OO,EH_n,N,kset,ES_sectors,pow,evR_set_grouped,euR_pow,evL_set_grouped,SPIN_eig_set_grouped,DTrun_FLR_eig,mpo_type)

elseif Dtrun_method=="svds"
    DTrun=length(group_numbers(SPIN_svd_set));
    display("DTrun="*string(DTrun));

    Ss=abs.(group_numbers(S_set));
    Trun_err=(minimum(Ss)/maximum(Ss));

    kset,Eset=solve_ITEBD_excitation_TrunTransOp_iterative(Ag,O1,O2,OO,EH_n,N,kset,ES_sectors,pow,U_set_grouped,S_set_grouped,Vh_set_grouped,SPIN_svd_set_grouped,DTrun_FLR_svd,mpo_type)
end

ES_filenm="ES_"*Dtrun_method*"_D"*string(D)*"_chi"*string(chi)*"_W"*string(W)*"_N"*string(N)*".mat";
matwrite(ES_filenm, Dict(
    "kset" => convert(Vector,kset),
    "ES_sectors" => ES_sectors,
    "Eset" => Eset,
    "Trun_err"=>Trun_err,
    "DTrun"=>DTrun
); compress = false)


"kset=0:5"

"DTrun=91"

"sector k=0, spin=0.0:"

5-element Vector{ComplexF64}:
  0.9489266550439872 - 0.5505849050882771im
  0.9447189847639647 + 0.5479755565396066im
 0.17041675382305294 - 0.08412249496841885im
 0.17042837540620284 + 0.08407879234171813im
 0.05042665534779483 - 2.5998111817538736e-6im

"sector k=0, spin=0.5:"

...

"sector k=5, spin=2.0:"

"MPS decomposition does not have this sector, skip it"

"sector k=5, spin=2.5:"

"MPS decomposition does not have this sector, skip it"

calculate ES for N=6 73.558799 seconds (158.14 M allocations: 8.736 GiB, 5.92% gc time, 98.82% compilation time)
  1.900995 seconds (5.63 M allocations: 640.787 MiB, 8.07% gc time, 19.91% compilation time)
  1.147719 seconds (3.52 M allocations: 421.705 MiB, 13.93% gc time, 19.10% compilation time)
  1.076286 seconds (3.18 M allocations: 380.227 MiB, 7.41% gc time, 23.35% compilation time)
  0.735780 seconds (1.56 M allocations: 170.296 MiB, 21.34% gc time, 12.25% compilation time)
  1.664588 seconds (3.96 M allocations: 449.308 MiB, 21.89% gc time)
  0.622700 seconds (1.84 M allocations: 211.550 MiB, 7.94% gc time)
  0.577905 seconds (1.78 M allocations: 205.446 MiB, 3.64% gc time)
  0.522887 seconds (1.50 M allocations: 167.024 MiB, 11.07% gc time)
  1.363585 seconds (3.96 M allocations: 449.308 MiB, 6.88% gc time)
  0.647049 seconds (1.84 M allocations: 211.550 MiB, 6.77% gc time)
  0.673745 seconds (1.78 M allocations: 205.446 MiB, 5.78% gc time)
  0.484721 seconds (1.50 M allocati

In [12]:
Dtrun_method="svds";
ES_sectors=[0,1/2,1,3/2,2,5/2];
kset=0:N-1;
#kset=0:0
Eset=[];
Trun_err=0;
DTrun=0;
print("calculate ES for N="*string(N));
display("kset="*string(kset));
pow=round((N-2)/2);
k=0;

norm_eff=excitation_TrunTransOp_iterative_norm_eff(Ag,pow,N,k) # put it on cpu because this matrix maybe large
norm_eff=permute(norm_eff,(1,2,3,),(4,5,6,))
uu,ss,vvt=tsvd(norm_eff, trunc=truncerr(0.0000001));
norm_eff=[]#clear this big matrix
input_transform=vvt';
output_transform=uu';
output_transform=output_transform;
output_transform=pinv(ss)*output_transform;

sector_ind=1;
SPIN=ES_sectors[sector_ind];
sectr=Irrep[SU₂](SPIN);

v_init=TensorMap(randn,domain(input_transform), SU₂Space(SPIN=>1));



"kset=0:19"

calculate ES for N=20

TensorMap(ProductSpace(Rep[SU₂](0=>101, 1/2=>166, 1=>178, 3/2=>146, 2=>96, 5/2=>50, 3=>20, 7/2=>6, 4=>1)) ← ProductSpace(Rep[SU₂](0=>1))):
* Data for fusiontree FusionTree{Irrep[SU₂]}((0,), 0, (false,), ()) ← FusionTree{Irrep[SU₂]}((0,), 0, (false,), ()):
  0.5977862641628349
  0.35938832363934153
 -0.9415609128457835
  0.11470704942783432
 -0.6420393992645947
  1.8453080111415037
  1.3949468553356867
 -0.027372549490894262
 -0.9803691135501182
  0.7659139583364811
  ⋮
  0.7474111029900774
  0.22957864369938505
  0.287035435345438
 -0.12061492314130684
  0.04387291750859652
  0.27760426559275647
  1.2807173454482943
 -0.42095669490152876
 -0.08149813294412565


In [15]:

mpo_type="O_O"
@time excitation_TrunTransOp_iterative_H_eff(v_init,input_transform,output_transform,O1,O2,OO,Ag,pow,U_set_grouped,S_set_grouped,Vh_set_grouped,SPIN_svd_set_grouped,N,k,DTrun_FLR_svd,mpo_type)



 86.721520 seconds (124.22 M allocations: 22.046 GiB, 22.17% gc time)

TensorMap(ProductSpace(Rep[SU₂](0=>101, 1/2=>166, 1=>178, 3/2=>146, 2=>96, 5/2=>50, 3=>20, 7/2=>6, 4=>1)) ← ProductSpace(Rep[SU₂](0=>1))):
* Data for fusiontree FusionTree{Irrep[SU₂]}((0,), 0, (false,), ()) ← FusionTree{Irrep[SU₂]}((0,), 0, (false,), ()):
    0.9642210530953496 + 0.007727461427083748im
  -0.14194476820733104 + 0.044212789577702916im
   -0.7011437407816743 - 0.047127496123491375im
  -0.14562798758644263 - 0.11589631320665683im
  -0.25523617663833703 - 0.0031056406722010377im
     0.180586014406656 + 0.007679189642388645im
 -0.042981508141303915 - 0.0011401608713610817im
 -0.008382299299796128 + 0.0037374838175143754im
   0.01924312112397114 + 0.003020142481723794im
   0.36571955627513614 + 0.0024478431354699737im
                       ⋮
  -0.16907366463000884 + 0.03272759098591127im
    -1.093397733573994 + 0.42899427222881314im
  0.032119892130520356 + 2.168294295672169im
     -2.11775507035361 + 0.7734135651854501im
   0.35284338162077233 - 0.21406092197523208im
    